In [10]:
import pandas as pd
import numpy as np
import pickle

df = pickle.load(open('Beer_Rating_Dataset.pkl','rb'))

In [11]:
df.head()

,beer,brewery,rDev,rating,username,beer_id
0,Barrel Aged Lucifer's Cuvee,Latitude 42 Brewing Company,-10.4,3.52,stonedtrippin,65265
1,BBA Ye Old Battering Ram,MadTree Brewing,2.6,4.30,stonedtrippin,68664
2,Time Crystal,Cerebral Brewing,3.4,4.25,stonedtrippin,29236
3,Tin Cup,Burial Beer Co.,-4.5,3.81,stonedtrippin,25711
4,La Folie Grand Reserve: Geisha Sour Ale,New Belgium Brewing,0.0,4.31,stonedtrippin,77009


In [12]:
a = set(df.beer.values); b = set(df.brewery.values)
WORDS = a|b

In [13]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

#WORDS = Counter(words(open('big.txt').read()))

#def P(word, N=sum(WORDS.values())): 
#    "Probability of `word`."
#    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in split`bs if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

SyntaxError: invalid syntax (<ipython-input-13-0f5e4f3b91a3>, line 30)

In [14]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
df.brewery

In [15]:
Words = list(set(df.brewery.values))
brewers = [x.replace(' Brewing','').replace(' Brewery','').replace(' Company','').replace(' Co.','').replace(' Tasting Room','').replace(' Pub','').replace(' &','') for x in Words]
r = [nlp(x) for x in Words if type(x)==unicode]
rc = [nlp(x) for x in brewers if type(x)==unicode]

In [ ]:
def Brewery_Lookup(entry):
    enter = nlp(entry)
    simil = []
    for k in rc:
        f = enter.similarity(k)
        simil.append(f)
    ind = np.argsort(simil)[-1::-1]
    if (simil[0] != simil[1]):
        return r[ind[0]]
    else:
        return LookUp(entry)

In [ ]:
Wurds = list(set(df.beer.values))
q = [nlp(x) for x in Wurds if type(x)==unicode]

In [ ]:
def Beer_Lookup(entry):
    enter = nlp(entry)
    simil = []
    for k in q:
        f = enter.similarity(k)
        simil.append(f)
    ind = np.argsort(simil)[-1::-1]
    if (simil[0] != simil[1]):
        print r[ind[0]]
    else:
        print 'error'

In [ ]:
Brewery_Lookup(u'Balast')

In [ ]:
import nltk
nltk.edit_distance(u"Tröegs", u"Trogs")

In [ ]:
p = u'Ballast Brewing'
p.replace(' Brewing','')

In [ ]:
# clean up code with extra list comprehension
def LookUp_Brewery(entry):
    brewers = [x.replace(' Brewing','').replace(' Brewery','').replace(' Company','').replace(' Co.','').replace(' Tasting Room','').replace(' Pub','').replace(' &','').replace(' Craft','').replace(' Brauerei','').replace('Berkeley','') for x in Words]
    entered = entry.replace(' Brewing','').replace(' Brewery','').replace(' Company','').replace(' Co.','').replace(' Tasting Room','').replace(' Pub','').replace(' &','').replace(' Craft','').replace(' Brauerei','').replace('Berkeley','')
    f = [[nltk.edit_distance(entered,brewers[x]),Words[x]] for x in range(len(brewers)) if entered[0]==brewers[x][0]]
    return [sorted(f)[0][1], sorted(f)[1][1], sorted(f)[2][1]]

In [ ]:
#token words, split on spaces, find search based on each word and created composite score for phrase
LookUp_Brewery(u'Coors')

In [ ]:
dog = ' sentence about a dog Brewery Brewing Pub'
import re
m = re.findall(r'\b\w+\b',' sentence about a dog')
m

In [ ]:
def Super_Lookup(entry):
    #remove stopwords
    stopwords = [' Brewing', ' Brewery',' Company',' Co.',' Tasting Room',' Pub',' &',' Craft',' Brauerei','Berkeley','Works','Co']
    brewers = Words
    for r in stopwords:
        entry = entry.replace(r,'')
        brewers = [x.replace(r,'') for x in brewers]
    
    #tokenize entry and brewers
    entry_len =  len(re.findall(r'\b\w+\b', entry))
    brewer_len = {brewers[x]:(len(re.findall(r'\b\w+\b', brewers[x])),x) for x in range(len(brewers))}
    poss = [(x, brewer_len[x][1]) for x in brewer_len.keys() if brewer_len[x][0]>=entry_len]
    #return closest match with same number of words
    f = [[nltk.edit_distance(entry,poss[x][0]),Words[poss[x][1]]] for x in range(len(poss))]
    return [sorted(f)[0][1], sorted(f)[1][1], sorted(f)[2][1]]

In [ ]:
Super_Lookup('BarrelHouse')

In [ ]:
b = {'a':(1,2),'b':'4'}
b['a'][1]

In [ ]:
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer

schema = Schema(beer_id=ID(stored=True),
                beer=TEXT(stored=True),
                
                body=TEXT(analyzer=StemmingAnalyzer()),
                tags=KEYWORD)

In [ ]:
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer

schema = Schema(from_addr=ID(stored=True),
                to_addr=ID(stored=True),
                subject=TEXT(stored=True),
                body=TEXT(analyzer=StemmingAnalyzer()),
                tags=KEYWORD)